In [ ]:
#data path
imgs_dir = "../input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/"

label_dir  = "../input/diabetic-retinopathy-224x224-gaussian-filtered/train.csv"


In [ ]:
#gpu 
!nvidia-smi

In [ ]:
#importing required packages
import pandas as pd
from PIL import Image
import numpy as np
import os

In [ ]:
#reading train.csv 
labels = pd.read_csv(label_dir)
labels.head()

In [ ]:
#collecting the imgs path
import random 
imgs_path = []
for fol in os.listdir(imgs_dir):
    if fol != 'export.pkl':
        for file in os.listdir(imgs_dir+fol+'/'):
            imgs_path.append(imgs_dir+fol+'/'+file)
#shuffle the imgs path inorder to avoid overfit and underfit condition
random.shuffle(imgs_path)
imgs_path[:5]

In [ ]:
#labelling
target_labels = []
img_ids = list(labels['id_code'])
dia = list(labels['diagnosis'])
for file in imgs_path:
    _id = file.split('/')[-1].split('.')[0]
    ind = img_ids.index(_id)
    target_labels.append(dia[ind])
target_labels[:5]
    

In [ ]:
import cv2
from matplotlib import pyplot as plt
  
# create figure
fig = plt.figure(figsize=(20, 20))

i=0
for file in imgs_path[0:10]:
    i+=1
    fig.add_subplot(5, 5, i)
    img = cv2.imread(file,cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.title(file.split('/')[-2])


In [ ]:
from tensorflow.keras.utils import to_categorical
target_labels = to_categorical(target_labels,5)

In [ ]:
#loading imgs
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
data=[]
for img in imgs_path:
    image = load_img(img, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)

In [ ]:
#spliting data
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(data,target_labels,test_size=0.2,random_state = 51,stratify = target_labels)

In [ ]:
#numpyConverting
x_train = np.array(x_train, dtype="float32")
y_train = np.array(y_train)
x_val = np.array(x_val, dtype="float32")
y_val = np.array(y_val)


In [ ]:
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(rotation_range=20,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode="nearest")

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam

base_model= MobileNetV2(input_shape=(224,224,3), weights='imagenet', include_top=False)
x = base_model.output
x = Flatten()(x)
x = Dense(64,kernel_initializer='he_uniform')(x)
x = Activation('relu')(x)
x = Dense(128,kernel_initializer='he_uniform')(x)
x = Activation('relu')(x)
x = Dense(256,kernel_initializer='he_uniform')(x)
x = Activation('relu')(x)
x = Dense(128,kernel_initializer='he_uniform')(x)
x = Activation('relu')(x)
x = Dense(64,kernel_initializer='he_uniform')(x)
x = Activation('relu')(x)
predictions = Dense(5, activation='softmax')(x)

model = Model(inputs=base_model.input,outputs=predictions)


In [ ]:
INIT_LR = 1e-4
EPOCHS = 250
BS = 10


In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
H = model.fit(aug.flow(x_train, y_train, batch_size=BS),
              steps_per_epoch=len(x_train) //(3* BS),
              validation_data=(x_val, y_val),
              validation_steps=len(x_val) // (3*BS),
              epochs=EPOCHS)

In [ ]:
from matplotlib import pyplot as plt
N = EPOCHS
plt.style.use("ggplot")
plt.figure(figsize=(15,10))
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(loc="center right")

In [ ]:
from matplotlib import pyplot as plt
N = EPOCHS
plt.style.use("ggplot")
plt.figure(figsize=(15,10))
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")

plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(loc="center right")

In [ ]:
print("Performance Report:")
y_pred=model.predict(x_val)
y_pred = [np.argmax(x) for x in y_pred]
y_test=[np.argmax(x) for x in y_val]

target=['0','1','2','3','4']
from sklearn import metrics
#print('Accuracy score is :', np.round(metrics.accuracy_score(y_test, y_pred),4))
print('Precision score is :', np.round(metrics.precision_score(y_test, y_pred, average='weighted'),4))
print('Recall score is :', np.round(metrics.recall_score(y_test,y_pred, average='weighted'),4))
print('F1 Score is :', np.round(metrics.f1_score(y_test, y_pred, average='weighted'),4))

print('\t\tClassification Report:\n', metrics.classification_report(y_test, y_pred,target_names=target))

In [ ]:
model.save('../output/model.h5')

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
#tflite conversion for size optimization
import tensorflow as tf
model = tf.keras.models.load_model('../output/model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("../output/model.tflite", "wb").write(tflite_model)